In [ ]:
# !pip install osm2geojson

In [ ]:
%load_ext autoreload
%autoreload 2


import requests
import json
import geopandas as gpd
import folium.plugins
import logging
from eo_tools.util import explore_products
from osm2geojson import json2geojson

from eodag import EODataAccessGateway
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)
from shapely.geometry import box


In [ ]:

# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the Overpass QL query
overpass_query = """
[out:json];
area["ISO3166-1"="FR"]->.countryArea;
relation["boundary"="administrative"]["admin_level"="4"](area.countryArea);
out body;
>;
out skel qt;
"""

# Make a request to the Overpass API
response = requests.post(overpass_url, data={'data': overpass_query})

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Optional: Pretty print the JSON response
    # print(json.dumps(data, indent=2))
else:
    print(f"Error: {response.status_code}")

In [ ]:
gj = json2geojson(data)
gdf = gpd.GeoDataFrame.from_features(gj)

In [ ]:
# TODO: cleaner way
aoi_bretagne = gdf[gdf['id']==102740].geometry[6]

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
shp = box(*aoi_bretagne.bounds)
search_criteria = {
    # "productType": "S2_MSI_L1C",
    "productType": "S1_SAR_SLC",
    "start": "2024-04-01",
    "end": "2024-04-16",
    "geom": shp
}
results = dag.search_all(**search_criteria)

In [ ]:
explore_products(results, aoi_bretagne)

In [ ]:
# dag.download_all(results[16:17], outputs_prefix="/data/S1/", extract=True)
dir_slc = '/data/S1/S1A_IW_SLC__1SDV_20240410T180524_20240410T180551_053375_067920_A0F7/S1A_IW_SLC__1SDV_20240410T180524_20240410T180551_053375_067920_A0F7.SAFE'

In [ ]:
from eo_tools.S1.process_single import process_slc

process_slc(
    dir_prm=dir_slc,
    outputs_prefix="/data/res/test-slc-proc",
    shp=aoi_bretagne,
    pol="vv",
    skip_preprocessing=True
)

In [ ]:
dir_out = '/data/res/test-slc-proc/S1_2024-04-10-180524'

In [ ]:
from eo_tools_dev.util import show_sar_int, show_cog
# show_sar_int(f"{dir_out}/amp_prm_vv.tif")
show_cog(f"{dir_out}/amp_prm_vv.tif", rescale="0,1", resampling="cubic", expression="log(b1+1)")